In [1]:
from tqdm import tqdm
import pandas as pd
import json
import requests
import time

In [2]:
bot_name = 'curekart_subset'

In [3]:
with open(f'keys/{bot_name}.json', 'r') as f:
    keys = json.load(f)

In [4]:
APP_ID = keys['app_id']
PREDICTION_KEY = keys['primary_key']
PREDICTION_ENDPOINT = keys['endpoint_url']

In [5]:
def predict_node_name(text):
        params = {
            'query': text,
            'timezoneOffset': '0',
            'verbose': 'true',
            'show-all-intents': 'true',
            'spellCheck': 'true',
            'staging': 'false',
            'subscription-key': PREDICTION_KEY
        }

        response = requests.get(f'{PREDICTION_ENDPOINT}luis/prediction/v3.0/apps/{APP_ID}/slots/production/predict', headers={}, params=params)

        predicted_node = response.json()['prediction']['topIntent']
        predicted_node_score = response.json()['prediction']['intents'][predicted_node]['score']

        return predicted_node, predicted_node_score

In [6]:
test_file_name = bot_name if '_subset' not in bot_name else bot_name.replace('_subset', '')
df_test = pd.read_csv(f'../../test/{test_file_name}_test.csv')
df_test.head()

,sentence,label
0,Order my product,ORDER_TAKING
1,Hy,NO_NODES_DETECTED
2,I want to order Wottle sensitive handwash refill,RECOMMEND_PRODUCT
3,have u started C D provision for pin code 702164,CHECK_PINCODE
4,How do I know it will deliver in my place,NO_NODES_DETECTED


In [7]:
df_test.shape

(991, 2)

In [8]:
predict_node_name('Order my product')

('ORDER_STATUS', 0.131283209)

In [9]:
pred_nodes = []
pred_scores = []
for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    node_name, node_score = predict_node_name(row['sentence'])
    pred_nodes.append(node_name)
    pred_scores.append(node_score)
    time.sleep(0.3)
df_test['predicted_node'] = pred_nodes
df_test['predicted_node_score'] = pred_scores
df_test.head()

100%|██████████| 991/991 [19:58<00:00,  1.21s/it]


,sentence,label,predicted_node,predicted_node_score
0,Order my product,ORDER_TAKING,ORDER_STATUS,0.131283
1,Hy,NO_NODES_DETECTED,RECOMMEND_PRODUCT,0.457261
2,I want to order Wottle sensitive handwash refill,RECOMMEND_PRODUCT,RECOMMEND_PRODUCT,0.042601
3,have u started C D provision for pin code 702164,CHECK_PINCODE,RESUME_DELIVERY,0.077872
4,How do I know it will deliver in my place,NO_NODES_DETECTED,RESUME_DELIVERY,0.943636


In [10]:
set(df_test['predicted_node']) - set(df_test['label'])

{'IMMUNITY', 'INTERNATIONAL_SHIPPING', 'None', 'PORTAL_ISSUE'}

In [11]:
pred_nodes = []
for index, row in df_test.iterrows():
    if row['predicted_node'] == 'None':
        pred_nodes.append('NO_NODES_DETECTED')
    else:
        pred_nodes.append(row['predicted_node'])
df_test['predicted_node'] = pred_nodes
print(set(df_test['predicted_node']) - set(df_test['label']))

{'PORTAL_ISSUE', 'INTERNATIONAL_SHIPPING', 'IMMUNITY'}


In [12]:
df_test.head()

,sentence,label,predicted_node,predicted_node_score
0,Order my product,ORDER_TAKING,ORDER_STATUS,0.131283
1,Hy,NO_NODES_DETECTED,RECOMMEND_PRODUCT,0.457261
2,I want to order Wottle sensitive handwash refill,RECOMMEND_PRODUCT,RECOMMEND_PRODUCT,0.042601
3,have u started C D provision for pin code 702164,CHECK_PINCODE,RESUME_DELIVERY,0.077872
4,How do I know it will deliver in my place,NO_NODES_DETECTED,RESUME_DELIVERY,0.943636


In [13]:
df_test.to_csv(f'../../preds/luis_{bot_name}.csv', index=False)

In [14]:
df_test.shape

(991, 4)